In [1]:
import numpy as np
from scipy.stats import uniform
from scipy.stats import norm

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import os

import PK_MONDclass as MONDclass

In [2]:
gplus = 1.2 * 10 ** -10
MONDmodel = MONDclass.MOND(gplus)

In [3]:
sampleObsInfo = pd.read_csv('sampleObsInfo135.csv', 
                            names=['GalaName','HubbleType','Distance','DisErr',
                                   'DisMethod','Inclination','inclinationErr','totLum3p6','LumErr',
                                   'Reff','SurfBeff','Rd',
                                   'CentSurfB','M_HI','R_HI',
                                   'Vf','VfErr','Q','Ref'],
                            index_col = 'GalaName')

In [4]:
galaname = sampleObsInfo.index.values
galaname.shape

(135,)

In [5]:
##################################################
# make function for different velocities

def v_baryon(M7L_disk, M7L_bulge=0):
    return np.sqrt(M7L_disk*V_disk**2 + M7L_bulge*V_bulge**2 + V_gas**2)

def v_disk(M7L_disk):
    return np.sqrt(M7L_disk) * V_disk

def v_total(R_gala, M7L_disk, M7L_bulge=0):
    vbar = v_baryon(M7L_disk, M7L_bulge)
    return MONDmodel.v_obs(vbar, R_gala)

# chi2 per dof
def calc_chi2dof(M7L_disk, M7L_bulge=0):
    vtot = v_total(R_gala, M7L_disk, M7L_bulge)
    chi2 = np.sum(((vtot - V_obs)**2 / V_errV**2))
    return 1 / dof * chi2

##################################################
# make function for prior

def make_uniform_prior(start, width):
    def inner(x):
        return uniform.pdf(x, start, width)
    return inner

# prior_M7Ldisk_unif = make_uniform_prior(-1, 1-(-1))
# prior_M7Lbulge_unif = make_uniform_prior(-1, 1-(-1))

prior_M7Ldisk_unif = make_uniform_prior(0.1, 10-(0.1))
prior_M7Lbulge_unif = make_uniform_prior(0.1, 10-(0.1))

def make_norm(mean, sd):
    def inner(x):
        return norm.pdf(x, mean, sd)
    return inner

prior_M7Ldisk_norm = make_norm(np.log10(0.5), 0.1)
prior_M7Lbulge_norm = make_norm(np.log10(0.7), 0.1)

In [6]:
# theta = [M7Ldisk]

def lnprior(theta):
    M7L_disk, M7L_bulge = theta
    if (0.1 <= M7L_disk <= 10) and (0.1 <= M7L_bulge <= 10):
        prior = prior_M7Ldisk_unif(M7L_disk) * prior_M7Lbulge_unif(M7L_bulge)
        return np.log10(prior)
    else:
        return -np.inf
    
def lnlike(theta):
    M7L_disk, M7L_bulge = theta
    vtot = v_total(R_gala, M7L_disk, M7L_bulge)
    like = -1/2 * np.sum(((vtot - V_obs)**2 / V_errV**2))
    return like

def lnprob(theta):
    lp = lnprior(theta)
#     print(lp)
    if not np.isfinite(lp):
        return -np.inf
#     if lp == np.log(10**-100):
#         return lp
    return lp + lnlike(theta)

In [7]:
ndim, nwalkers = 2, 10

In [8]:
import emcee
import corner
from tqdm import tqdm

In [9]:
file_name = ['chain', 'chain_plot', 'corner_plot', 'fits_plot']
for name in file_name:
    try:
        os.mkdir('./Output_MOND/'+name)
    except FileExistsError:
        pass    

f = open('./Output_MOND/best_para_bulge.txt','w')
for kk in tqdm(range(135)):
    name = galaname[kk]
    data = np.loadtxt('./SPARC/Rotmod/'+name+'_rotmod.dat', delimiter='\t', skiprows=2)
    Ndata = len(data)
    
    R_gala = data[:,0]
    Rmax = max(R_gala)

    V_obs = data[:,1]
    V_errV = data[:,2]
    Vobsmax = max(V_obs + V_errV)
    
    V_gas = data[:,3]
    V_disk = data[:,4]
    V_bulge = data[:,5]
    YN_bulge = any(V_bulge) 
    
    if YN_bulge:
        dof = Ndata - 2
    else:
        dof = Ndata - 1
        
    if not YN_bulge:
        continue
    
    pos = [np.array([4,6]) + 3* np.random.random_sample(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)
    sampler.run_mcmc(pos, 5000)
    mcmcchain = sampler.chain
    
    # output the chain data
    directory = './Output_MOND/chain/'+name
    
    try:
        os.mkdir(directory)
    except FileExistsError:
        pass
    
    for i in range(nwalkers):
        np.savetxt('./Output_MOND/chain/'+name+'/'+name+'_chain_part'+str(i)+'.csv',mcmcchain[i])
        
    ##################################
    
    samples = sampler.chain.reshape((-1,ndim))
    fig_corner = corner.corner(samples, range=[0.95, 0.95], labels=["M/L_D", "M/L_B"])
    fig_corner.savefig('./Output_MOND/corner_plot/'+name+'_corner.pdf')
    plt.close(fig_corner) 
    
    ##################################
    
#     fig_chain, axes = plt.subplots(figsize=(8, 8))
#     labels = ["M/L_D"]
#     for i in range(ndim):
#         ax = axes
#         ax.plot(mcmcchain[:, :, i].T, "k", alpha=0.3)
#         ax.set_xlim(0, 5000)
        
#         ax.set_ylabel(labels[i])
#     axes.set_xlabel("step number")
    
#     fig_chain.savefig('./Output_MOND/chain_plot/'+name+'_chain.pdf')
#     plt.close(fig_chain) 
    
    ##################################
    
    post_prob = list(map(lnprob, samples))
    
    mid = 50 # middle value
    dev = 68.27 / 2 # deviation from the middle value
    postlow = np.percentile(post_prob, mid - dev, interpolation='lower')
    postmid = np.percentile(post_prob, mid, interpolation='lower')
    posthigh = np.percentile(post_prob, mid + dev, interpolation='lower')
    postmax = np.max(post_prob)
#     postlow, postmid, posthigh, postmax

    posilow = np.argwhere(post_prob == postlow)[0,0]
    posimid = np.argwhere(post_prob == postmid)[0,0]
    posihigh = np.argwhere(post_prob == posthigh)[0,0]
    posimax = np.argwhere(post_prob == postmax)[0,0]
#     posilow, posimid, posihigh, posimax

    M7Lmax = samples[posimax]
    minchi2dof = calc_chi2dof(M7Lmax[0], M7Lmax[1])
#     M7Lmax, minchi2dof, dof

    f.write('%s %f %f %f %f\n' % (name, M7Lmax[0], M7Lmax[1], minchi2dof, dof))
    
    ##################################
    
    fig_chain, axes = plt.subplots(2, figsize=(10, 7))
    labels = ["M/L_D", "M/L_B"]
    for i in range(ndim):
        ax = axes[i]
        ax.plot(mcmcchain[:, :, i].T, "k", alpha=0.3)
        ax.set_xlim(0, 5000)
        ax.set_ylim(M7Lmax[i]/2,M7Lmax[i]*2)
        ax.set_ylabel(labels[i])
    axes[-1].set_xlabel("step number")
    
    fig_chain.savefig('./Output_MOND/chain_plot/'+name+'_chain.pdf')
    plt.close(fig_chain) 
    
    ##################################
    
    M7Llow, M7LlowB = samples[posilow]
    M7Lhigh, M7LhighB = samples[posihigh]

    vtot_low = v_total(R_gala, M7Llow, M7LlowB)
    vtot_high = v_total(R_gala, M7Lhigh, M7LhighB)
    vtotmaxlow = np.max(vtot_low)
    vtotmaxhigh = np.max(vtot_high)

    vbar_low = v_baryon(M7Llow, M7LlowB)
    vbar_high = v_baryon(M7Lhigh, M7LhighB)

    vdisk_low = v_disk(M7Llow)
    vdisk_high = v_disk(M7Lhigh)
        
    #################################
    fig_RC, ax = plt.subplots(figsize=(6,6))
    ax.errorbar(R_gala, V_obs, yerr=V_errV, fmt='ko', ecolor='k', capsize=3.5, markersize=4)
    # ax.plot(R_gala, vtot_test, 'ro',
    #         R_gala, V_disk, 'y*',
    #         R_gala, V_gas, 'g^',
    #         R_gala, V_bulge, 'c*')
            #R_gala, V_nfw, 'mo')
    # ax.plot(R_gala, vtot_test, 'ro')

    ax.plot(R_gala, vbar_low, color=(0,0.3,0), alpha=0.8)
    ax.plot(R_gala, vbar_high, color=(0,0.3,0), alpha=0.8)
    ax.plot(R_gala, vdisk_low, color=(0.7,0.7,0.7), alpha=0.8)
    ax.plot(R_gala, vdisk_high, color=(0.7,0.7,0.7), alpha=0.8)
    ax.plot(R_gala, vtot_low, color=(1,0,0), alpha=0.8)
    ax.plot(R_gala, vtot_high, color=(1,0,0), alpha=0.8)    
    

    ax.fill_between(R_gala, vbar_low, vbar_high, facecolor=(0,0.3,0), alpha=0.8, label='Baryons')
    ax.fill_between(R_gala, vdisk_low, vdisk_high, facecolor=(0.7,0.7,0.7), alpha=0.8, label='Disk')
    ax.fill_between(R_gala, vtot_low, vtot_high, facecolor=(1,0,0), alpha=0.8, label='Model')

    ax.set_xlim(0,Rmax*1.05)
    ax.set_ylim(0,np.max([Vobsmax,vtotmaxlow,vtotmaxhigh])*1.1)
    ax.set_xlabel('radius (kpc)',fontsize=15)
    ax.set_ylabel('rotation speed (km/s)',fontsize=15)
    ax.tick_params(width=1.5, labelsize=15)
    ax.set_title(name,fontsize=15)
    ax.legend(loc='lower right',fontsize=12)

    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1.)

    fig_RC.savefig('./Output_MOND/fits_plot/'+name+'_fits.pdf', bbox_inches='tight')
    plt.close(fig_RC) 
    
f.close()   
    

100%|██████████| 135/135 [16:40<00:00,  7.41s/it]


In [10]:
np.max([1,2,3])

3